# **Fundamentals of Data Science - Winter Semester 2023**


#### Prof. Fabio Galasso, Stefano D'Arrigo (TA), Edoardo De Matteis (TA), Daniele Trappolini (TA)
<galasso@di.uniroma1.it>, <darrigo@di.uniroma1.it> , <dematteis@di.uniroma1.it>, <daniele.trappolini@uniroma1.it>

## **#2 Homework: Classification**


-----------------------------------

*Note: your task is to fill in the missing code where you see `"YOUR CODE HERE"` and the text part `"WRITE YOUR TEXT HERE"` part corresponding to each subproblem and produce brief reports on the results whenever necessary.*

As part of the homework, provide the answer to questions in this notebook report-like manner.

After you have implemented all the missing code in the required sections, you will be able to run all the code without any errors.

We kindly ask you to double-check this since **all** the delivered homework will be executed.

The completed exercise should be handed in as a single notebook file. Use Markdown to provide equations. Use the code sections to provide your scripts and the corresponding plots.

-------------------------------------

**Submit it** by sending an email to:

**galasso@di.uniroma1.it** , **darrigo@di.uniroma1.it**, **dematteis@di.uniroma1.it**, and **daniele.trappolini@uniroma1.it** **by Monday, November 27th, 23:59**.

-------------------------------------

**Outline and Scores for #2 Homework:**


* **Question 1: Logistic Resgression with Gradient Ascent** *(10 points)*
  * **Question 1.1: Log-likelihood and Gradient Ascent rule**
  * **Question 1.2: Implementation of Logistic Regression with Gradient Ascent**
  * **Question 1.3: Report**

* **Question 2: Logistic Regression with non linear boundaries** *(10 points)*
  * **Question 2.1: Polynomial features for logistic regression**
  * **Question 2.2: Plot the computed non-linear boundary**
  * **Question 2.3: Report**

* **Question 3: Multinomial Classification** *(10 points)*
  * **Question 3.1: Softmax Regression Model**
  * **Question 3.2: Coding**
  * **Question 3.3: Histogram Features Extraction**

* **BONUS Question 4: Transfer Learning on CIFAR-10** *(5 points)*
  * **Question 4.1: Train a KNN Classifier on CIFAR-10**
  * **Question 4.2: Train a Softmax Classifier on CIFAR-10**
  * **Question 4.3: Report**
  

**TOTAL POINTS ARE 35, BONUS QUESTION INCLUDED**

**Notation:**

- $x^i$ is the $i^{th}$ feature vector
- $y^i$ is the expected outcome for the $i^{th}$ training example
- $m$ is the number of training examples
- $n$ is the number of features

**Let's start by setting up our Python environment and importing the required libraries:**

In [ ]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn seaborn tqdm torch torchvision

In [ ]:
%matplotlib inline
import numpy as np # imports a fast numerical programming library
import scipy as sp # imports stats functions, amongst other things
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.cm as cm # allows us easy access to colormaps
import matplotlib.pyplot as plt # sets up plotting under plt
import pandas as pd # lets us handle data as dataframes
from sklearn.datasets import make_classification
from sklearn.preprocessing import normalize
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import os

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

import seaborn as sns # sets up styles and gives us more plotting options

## **Question 1: Logistic Regression with Gradient Ascent *(10 Points)***

### **Question 1.1: Log-likelihood and Gradient Ascent Rule *(4/10 Points)***

Write the likelihood $L(\theta)$ and log-likelihood $l(\theta)$ of the parameters $\theta$.

Recall the probabilistic interpretation of the hypothesis $h_\theta(x)= P(y=1|x;\theta)$ and that $h_\theta(x)=\frac{1}{1+\exp(-\theta^T x)}$.

Also derive the gradient $\frac{\delta l(\theta)}{\delta \theta_j}$ of $l(\theta)$ and write the gradient update equation.

-------------------------------------------------------

**WRITE YOUR EQUATIONS HERE**

- **Likelihood**:

\begin{align}
L(\theta) &= \prod_{i=1}^{m} P(y^{(i)} | x^{(i)};\theta) = \\
&= \prod_{i=1}^{m} h_{\theta}(x^{(i)})^{y^{(i)}} \cdot (1 - h_{\theta}(x^{(i)}))^{(1-y^{(i)})}
\end{align}


- **Log-Likelihood**:

\begin{align}
l(\theta) &= \sum_{i=1}^{m}\left\{y^{(i)} \log h_{\theta}(x^{(i)}) + (1-y^{(i)}) \log (1 - h_{\theta}(x^{(i)}))\right\}
\end{align}


- **Gradient of log-likelihood** (slide 5 p. 20):
\begin{align}
\frac{\delta l(\theta)}{\delta \theta_j} &= \
\sum_{i=0}^{m}\left[y^{(i)} - h_{\theta}(x^{(i)}))\right] \cdot x_{j}^{(i)}
\
\end{align}

----------------------------------attenti a queste due formule se vanno invertiti
\begin{align}
y^{(i)} - h_{\theta}(x^{(i)})
\end{align}
------------------------------------------------------ciao-----------------------------
- **Gradient update equation**:
For  $j=0,...,n$:
\begin{equation}
\theta_j = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) \cdot x_j^{(i)}
\end{equation}



-------------------------------------------------------

### **Question 1.2: Implementation of Logistic Regression with Gradient Ascent *(4/10 Points)***

Translate the equations you wrote above in code to learn the logistic regression parameters, $x^{(i)}_1$ and $x^{(i)}_2$ represent the two features for the $i$-th data sample $x^{(i)}$ and $y^{(i)}$ is its ground truth label.
The dataset used here is a customer service airline dataset, even if it is a toy dataset you will encounter some problems typical of real data.


**Do not write below this line just run it**

--------------------------------------------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/università/FDS/Homework2/HW2/data/Invistico_Airline.csv")
df.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0


In [ ]:
print(df["satisfaction"].value_counts())
print("-" * 30)
print(df["Customer Type"].value_counts())
print("-" * 30)
print(df["Type of Travel"].value_counts())
print("-" * 30)
print(df["Class"].value_counts())

We can see how some values are not numerical, we first explore the data counting the number of unique values for each feature.
Then, we replace the categorical values with numerical ones.

In doing that, we also deal with null values, there are many ways to deal with null values, among which:
1. Replace with a default value.
2. Replace them with the most frequent value for that feature.
3. Replace them with the mean or median value for that feature.

We opt for the mean value for each feature.

In [ ]:
# replace values with a number
df["satisfaction"] = df["satisfaction"].replace({"satisfied": 1, "dissatisfied": 0})
df["Customer Type"] = df["Customer Type"].replace(
    {"Loyal Customer": 1, "disloyal Customer": 0}
)
df["Type of Travel"] = df["Type of Travel"].replace(
    {"Business travel": 0, "Personal Travel": 1}
)
df["Class"] = df["Class"].replace({"Business": 0, "Eco": 1, "Eco Plus": 2})

# fill NaN values with mean
df["Arrival Delay in Minutes"].fillna(
    df["Arrival Delay in Minutes"].mean(), inplace=True
)
df.head()

It is advisable to look at the distribution of the data you will be dealing with.

In [ ]:
# Plot histograms for each variable
df.hist(figsize=(15, 15), bins=20)
plt.show()

In [ ]:
df_small = df[["Flight Distance", "Leg room service", "satisfaction"]]
df_small.head()

Separate features from labels.

In [ ]:
X, y = (df_small.drop(["satisfaction"], axis=1).values, df_small["satisfaction"].values)

It is recommended to normalize data when using gradient descent, we aim to have the data with mean $\mu=0$ and $\sigma=1$.

In [ ]:
X = (X - X.mean()) / X.std()

In [ ]:
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)

We add a column of 1's to $X$ to take into account the zero intercept.

In [ ]:
x = np.hstack([np.ones((X.shape[0], 1)), X])

Show the first and last 5 lines of $X$, now containing features $x_0$ (constant $1$), $x_1$ and $x_2$.

In [ ]:
[
    x[:5, :],
    x[-5:, :],
]

Show the first and last 5 lines of y

In [ ]:
[y[:5], y[-5:]]

Define the sigmoid function `sigmoid`, the function to compute the gradient of the log likelihood  `grad_l` and the gradient ascent algorithm.

*Hint: even though by definition log likelihood and gradient ascent are defined by summations, for numerical stability it is advised to use the mean operation.*


**Write your code below this line**

--------------------------------------------

In [ ]:
def sigmoid(x):
    """
    Function to compute the sigmoid of a given input x.

    Args:
        x: it's the input data matrix.

    Returns:
        g: The sigmoid of the input x
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return g


def log_likelihood(theta, x, y):
    """
    Function to compute the log likehood of theta according to data x and label y.

    Args:
        theta: it's the model parameter matrix.
        x: it's the input data matrix.
        y: the label array.

    Returns:
        log_l: the log likehood of theta according to data x and label y.
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return log_l


def predictions(features, theta):
    """
    Function to compute the predictions for the input features.

    Args:
        theta: it's the model parameter matrix.
        features: it's the input data matrix.

    Returns:
        preds: the predictions of the input features.
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return preds


def update_theta(theta, target, preds, feat, lr):
    """
    Function to compute the gradient of the log likelihood
    and then return the updated weights.

    Args:
        theta: the model parameter matrix.
        target: the label array.
        preds: the predictions of the input features.
        feat: it's the input data matrix.
        lr: the learning rate.

    Returns:
        theta: the updated model parameter matrix.
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return theta


def gradient_ascent(theta, feat, target, lr, num_steps):
    """
    Function to execute the gradient ascent algorithm.

    Args:
        theta: the model parameter matrix.
        target: the label array.
        num_steps: the number of iterations.
        feat: the input data matrix.
        lr: the learning rate.

    Returns:
        theta: the final model parameter matrix.
        log_likelihood_history: the values of the log likelihood during the process.
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return theta, log_likelihood_history


**Do not write below this line just run it**

--------------------------------------------

**Check your grad_l implementation:**

`grad_l` applied to the `theta_test` (defined below) should provide a value for `log_l_test` close to the `target_value` (defined below).
In other words, `error_test` should be 0, up to machine error precision.

In [ ]:
target_value = -1.1404441213525176
output_test = log_likelihood(np.array([-1, 0, 1]), x, y)
error_test = np.abs(output_test - target_value)
print("Error: ", error_test)

**Synthetic dataset**

Let's first test our implementation on a synthetic dataset.

In [ ]:
X_, y_ = make_classification(
    n_samples=500,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=1,
)
x_ = np.hstack([np.ones((X_.shape[0], 1)), X_])
x_.shape, y_.shape

In [ ]:
sns.scatterplot(x=X_[:, 0], y=X_[:, 1], hue=y_)

In [ ]:
# Initialize theta0
theta0 = np.random.normal(0, 0.01, x_.shape[1])

# Run Gradient Ascent method
n_iter = 50
theta_final, log_l_history = gradient_ascent(theta0, x_, y_, lr=0.5, num_steps=n_iter)

In [ ]:
fig, ax = plt.subplots(num=2)

ax.set_ylabel("l(Theta)")
ax.set_xlabel("Iterations")
_ = ax.plot(range(len(log_l_history)), log_l_history, "b.")

Let's plot the data and the decision boundary, you should see a line separating the two classes.

In [ ]:
# Plot raw data
sns.scatterplot(x=X_[:, 0], y=X_[:, 1], hue=y_)

# # Define range of x values
x_range = np.array([np.min(X_[:, 0]), np.max(X_[:, 0])])
y_range = -(theta_final[0] + theta_final[1] * x_range) / theta_final[2]
plt.plot(x_range, y_range, c="red")
# set the limits of the plot to the limits of the data
plt.xlim(np.min(X_[:, 0]) - 0.1, np.max(X_[:, 0] + 0.1))
plt.ylim(np.min(X_[:, 1]) - 0.001, np.max(X_[:, 1] + 0.001))
pass

**Real dataset**

Now we come back to the real dataset, we will see how things get harder in the real world.

We sample our $\theta_0$ from $\mathcal{N}(0,1e-2)$.

Let's apply the function gradient_ascent and print the final theta as well as theta_history:

In [ ]:
# Initialize theta0
theta0 = np.random.normal(0, 0.01, x.shape[1])

# Run Gradient Ascent method
n_iter = 50
theta_final, log_l_history = gradient_ascent(theta0, x, y, lr=0.5, num_steps=n_iter)

Let's plot the log likelihood over different iterations:


In [ ]:
fig, ax = plt.subplots(num=2)

ax.set_ylabel("l(Theta)")
ax.set_xlabel("Iterations")
_ = ax.plot(range(len(log_l_history)), log_l_history, "b.")

Plot the data and the decision boundary:

In [ ]:
# Plot raw data
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)

# # Define range of x values
x_range = np.array([np.min(X[:, 0]), np.max(X[:, 0])])
y_range = -(theta_final[0] + theta_final[1] * x_range) / theta_final[2]
plt.plot(x_range, y_range, c="red")
# set the limits of the plot to the limits of the data
plt.xlim(np.min(X[:, 0]) - 0.1, np.max(X[:, 0] + 0.1))
plt.ylim(np.min(X[:, 1]) - 0.001, np.max(X[:, 1] + 0.001))
pass

### **Question 1.3: Report *(2/10 Points)***

1. Are we looking for a local minimum or a local maximum using the gradient ascent rule?
2. You have implemented the gradient ascent rule. Could we have also used gradient descent instead for the proposed problem? Why/Why not?
3. Let's deeply analyze how the learning rate $\alpha$ and the number of iterations affect the final results. Run the algorithm you have written for different values of $\alpha$ and the number of iterations and look at the outputs you get. Is the decision boundary influenced by these parameters change? Why do you think these parameters are affecting/not affecting the results?
4. What happens if you do not normalize the data? Try to run the algorithm without normalizing the data and see what happens. Why do you think this happens?
5. We have choosen two features to train our model, but it does not mean that those are the best features to use. Try to use different features, at least two other pairs, and see if you can get a better or worse decision boundary, commenting the results you get.

-------------------------------------------------------


**WRITE YOUR ANSWER HERE:**

1.
2.
3.
4.
5. *(feel free to add here screenshots or new code cells if needed)*

-------------------------------------------------------

## **Question 2: Logistic Regression with non linear boundaries *(10 points)***

### **Question 2.1: Polynomial features for logistic regression *(4/10 Points)***

Define new features e.g., of $2-$nd and $3$-rd degree, and learn a logistic regression classifier by using the new features and the gradient ascent optimization algorithm defined in Question 1.

In particular, consider a polynomial boundary with equation:

\begin{equation}
f(x_1, x_2) = c_0 + c_1 x_1 + c_2 x_2 + c_3 x_1^2 + c_4 x_2^2 + c_5 x_1 x_2 + c_6 x_1^3 + c_7 x_2^3 + c_8 x_1^2 x_2 + c_9 x_1 x_2^2
\end{equation}

Therefore compute 7 new features: 3 new ones for the quadratic terms and 4 new ones for the cubic terms.

Create new arrays by stacking $x$ and the new 7 features (in the order $x_1x_1, x_2x_2, x_1x_2, x_1x_1x_1, x_2x_2x_2, x_1x_1x_2, x_1x_2x_2$).
In particular create `x_new_quad` by additionally stacking $x$ with the quadratic features, and `x_new_cubic` by additionally stacking $x$ with the quadratic and the cubic features.

**Do not write below this line just run it**

--------------------------------------------

In [ ]:
X, y = make_classification(
    n_samples=700,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=5,
)
X, X_test, y, y_test = train_test_split(X, y, test_size=200, random_state=42)

X.shape, y.shape

In [ ]:
x = np.hstack([np.ones((X.shape[0], 1)), X])
x_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

In [ ]:
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)

In [ ]:
# First extract features x1 and x2 from x and reshape them to x1 vector arrays
x1 = x[:, 1]
x2 = x[:, 2]
x1 = x1.reshape(x1.shape[0], 1)
x2 = x2.reshape(x2.shape[0], 1)
print(f"x:\n{x[:5, :]}\n{'-'*40}")  # For visualization of the first 5 values
print(f"x1:\n{x1[:5, :]}\n{'-'*40}")  # For visualization of the first 5 values
print(f"x2:\n{x2[:5, :]}")  # For visualization of the first 5 values

**Write your code below this line**

--------------------------------------------

In [ ]:
def get_polynomial(X, degree):
    """
    Given an initial set of features, this function computes the polynomial features up to the given degree.

    Args:
        X: the initial features
        degree: the degree of the polynomial

    Returns:
        X: the final polynomial features
    """
    if degree < 2:
        return X

    features = np.ones(X[:, 1].shape[0])

    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################
    return features


**Do not write below this line just run it**

--------------------------------------------

In [ ]:
x_new_quad = get_polynomial(x, degree=2)
x_new_cubic = get_polynomial(x, degree=3)
print(x_new_quad.shape, x_new_cubic.shape)

Now use the gradient ascent optimization algorithm to learn theta by maximizing the log-likelihood, both for the case of x_new_quad and x_new_cubic.

In [ ]:
# Initialize theta0, in case of quadratic features
theta0_quad = np.zeros(x_new_quad.shape[1])

theta_final_quad, log_l_history_quad = gradient_ascent(
    theta0_quad, x_new_quad, y, lr=0.5, num_steps=n_iter
)

# Initialize theta0, in case of quadratic and cubic features
theta0_cubic = np.zeros(x_new_cubic.shape[1])

# Run Newton's method, in case of quadratic and cubic features
theta_final_cubic, log_l_history_cubic = gradient_ascent(
    theta0_cubic, x_new_cubic, y, lr=0.5, num_steps=n_iter
)

# check and compare with previous results
print(theta_final_quad)
print(theta_final_cubic)

In [ ]:
# Plot the log likelihood values in the optimization iterations, in one of the two cases.
fig, ax = plt.subplots(num=2)

ax.set_ylabel("l(Theta)")
ax.set_xlabel("Iterations")
_ = ax.plot(range(len(log_l_history_quad)), log_l_history_quad, "b.")

### **Question 2.2: Plot the computed non-linear boundary *(4/10 Points)***



First, define a boundary_function to compute the boundary equation for the input feature vectors $x_1$ and $x_2$, according to estimated parameters theta, both in the case of quadratic (theta_final_quad) and of quadratic and cubic features (theta_final_cubic). Refer for the equation to the introductory part of Question 2.

**Write your code below this line**

--------------------------------------------

In [ ]:
def boundary_function(x1_vec, x2_vec, theta_final, degree):
    """
    This function computes the boundary function for the given theta_final and degree.

    Args:
        x1_vec: the x1 vector
        x2_vec: the x2 vector
        theta_final: the final theta
        degree: the degree of the polynomial

    Returns:
        x1_vec: the x1 vector
        x2_vec: the x2 vector
        f: the boundary function
    """

    x1_vec, x2_vec = np.meshgrid(x1_vec, x2_vec)

    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################

    return x1_vec, x2_vec, f


**Do not write below this line just run it**

--------------------------------------------

Now plot the decision boundaries corresponding to the `theta_final_quad` and `theta_final_cubic` solutions.

In [ ]:
def plot_boundary_function(
    X: np.ndarray, y: np.ndarray, theta: np.ndarray, degree: int, n_points: int = 200
) -> None:
    """
    This function plots the boundary function for the given theta and degree.

    Args:
        X: the input data
        y: the input labels
        theta: the final theta
        degree: the degree of the polynomial
        n_points: the number of points to plot

    Returns:
        None
    """

    x1_vec = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, n_points)
    x2_vec = np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, n_points)

    x1_vec, x2_vec, f = boundary_function(x1_vec, x2_vec, theta, degree=degree)
    mesh_shape = int(np.sqrt(f.shape[0]))

    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y)
    plt.contour(
        x1_vec, x2_vec, f.reshape((mesh_shape, mesh_shape)), colors="red", levels=[0]
    )
    plt.show()

In [ ]:
plot_boundary_function(X, y, theta_final_quad, degree=2)

In [ ]:
plot_boundary_function(X, y, theta_final_cubic, degree=3)

**Polynomial degree and overfitting**

As the polynomial degree increases, the decision boundary becomes more and more complex. This can lead to overfitting, i.e. the model learns the training data too well, and it is not able to generalize to new data. This is a common problem in machine learning, and it is important to be able to detect it.

In order to detect overfitting, we can split the dataset into a training set and a test set. The training set is used to learn the model, while the test set is used to evaluate the model performance on new data. If the model performs well on the training set, but it performs poorly on the test set, then we have overfitting.

In this exercise, you are asked to plot the training and test accuracy as a function of the polynomial degree. Consider all the polynomial degrees from 1 to 20. For each polynomial degree, learn the model on the training set, and evaluate the accuracy on both the training and the test set. Additionally, visualize the decision boundary for the polynomials that give the **best** and the **worst** test accuracy.

In [ ]:
def fit_polynomial(X, y, degree, lr, num_steps):
    """
    Given an initial set of features, this function computes the polynomial features up to the given degree.

    Args:
        X: the initial features
        y: the target values
        degree: the degree of the polynomial
        lr: the learning rate
        num_steps: the number of iterations

    Returns:
        theta_final: the final model parameter matrix
        log_l_history: the values of the log likelihood during the process
    """

    x_new = get_polynomial(X, degree=degree)

    # Initialize theta0
    theta0 = np.zeros(x_new.shape[1])

    # Run Gradient Ascent method
    theta_final, _ = gradient_ascent(theta0, x_new, y, lr=lr, num_steps=num_steps)

    return theta_final, x_new


def predict(x, theta):
    z = np.dot(x, theta)
    probabilities = sigmoid(z)
    y_hat = np.array(list(map(lambda x: 1 if x > 0.5 else 0, probabilities)))
    return y_hat


def fit_polynomials(X, y, X_test, y_test, degrees, lr, num_steps):
    """
    This function fits a logistic regression model for each degree in the degrees list.
    """
    X = normalize(X)
    X_test = normalize(X_test)

    thetas = []
    accuracy_scores_train, accuracy_scores_test = [], []
    for degree in tqdm(degrees):
        theta, x_new = fit_polynomial(X, y, degree, lr, num_steps)
        thetas += [theta]
        y_hat_train = predict(x_new, theta)
        accuracy_scores_train += [accuracy_score(y, y_hat_train)]
        y_hat_test = predict(get_polynomial(X_test, degree=degree), theta)
        accuracy_scores_test += [accuracy_score(y_test, y_hat_test)]

    return thetas, accuracy_scores_train, accuracy_scores_test

In [ ]:
degrees = np.arange(1, 12)
thetas, accuracy_scores_train, accuracy_scores_test = fit_polynomials(
    x, y, x_test, y_test, degrees=degrees, lr=0.5, num_steps=1000
)

In [ ]:
sns.lineplot(x=degrees, y=accuracy_scores_train, label="Train")
sns.lineplot(x=degrees, y=accuracy_scores_test, label="Test")
plt.xlabel("Degree")
plt.ylabel("Accuracy Score")
plt.xticks(degrees)
plt.show()

**Write your code below this line**

--------------------------------------------

Plot the best and the worst decision boundaries.

In [ ]:
# Plot worst model

In [ ]:
# Plot best model

### **Question 2.3: Report *(2/10 Points)***



Write now your considerations. Discuss in particular:
1. Look back at the plots you have generated. What can you say about the differences between the linear, quadratic, and cubic decision boundaries? Can you say if the model is improving in performances, increasing the degree of the polynomial? Do you think you can incur in underfitting increasing more and more the degree?
2. Look at the plot of the training and test accuracy as a function of the polynomial degree. What can you say about the differences between the training and test accuracy? What can you say about the differences between the best and the worst test accuracy? In general, is it desirable to have a very complex decision boundary, i.e. a very high degree of the polynomial? Discuss and motivate your answer.

-------------------------------------------------------


**WRITE YOUR ANSWER HERE:**

1.
2.



-------------------------------------------------------

## **Question 3: Multinomial Classification *(13 Points)***

### **Question 3.1: Softmax Regression Model *(4/10 Points)***

In the multinomial classification we generally have $K>2$ classes. So the label for the $i$-th sample $X_i$ is $y_i\in\{1,...,K\}$, where $i=1,...,N$. The output class for each sample is estimated by returning a score $s_i$ for each of the K classes. This results in a vector of scores of dimension K.
In this exercise we'll use the *Softmax Regression* model, which is the natural extension of *Logistic Regression* for the case of more than 2 classes. The score array is given by the linear model:

\begin{align*}
s_i =  X_i \theta
\end{align*}

Scores may be interpreted probabilistically, upon application of the function *softmax*. The position in the vector with the highest probability will be predicted as the output class. The probability of the class k for the $i$-th data sample is:

\begin{align*}
p_{ik} = \frac{\exp(X_i \theta_k)}{\sum_{j=1}^K(X_i \theta_j))}
\end{align*}

We will adopt the *Cross Entropy* loss and optimize the model via *Gradient Descent*.
In the first of this exercise we have to:
-    Write the equations of the Cross Entropy loss for the Softmax regression model;
-    Compute the equation for the gradient of the Cross Entropy loss for the model, in order to use it in the gradient descent algorithm.

#### A bit of notation

*  N: is the number of samples
*  K: is the number of classes
*  X: is the input dataset and it has shape (N, H) where H is the number of features
*  y: is the output array with the labels; it has shape (N, 1)
*  $\theta$: is the parameter matrix of the model; it has shape (H, K)

**Write you equation below this line**

--------------------------------------------

\begin{align*}
L(\theta) = ...
\end{align*}


\begin{align*}
\nabla_{\theta_k} L(\theta) = ...
\end{align*}


**Do not write below this line just run it**

--------------------------------------------

### **Question 3.2: Coding *(4/10 Points)***

Now we will implement the code for the equations. Let's implement the functions:
-  softmax
-  CELoss
-  CELoss gradient
-  gradient descent

We are using the CIFAR-10 dataset for this exercise. The CIFAR-10 dataset consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. It has 50,000 training images and 10,000 test images. The dataset was established by the Canadian Institute For Advanced Research (CIFAR), and it has become a standard benchmark for machine learning algorithms, especially in the area of image classification.

In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
test_data = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

*Hint: consider the labels as one-hot vector. This will allow matrix operations (element-wise multiplication and summation).*

In [ ]:
# Convert labels to one-hot encoded format
def one_hot_encode(y, num_classes=10):
    encoded = np.zeros((len(y), num_classes))
    for i, val in enumerate(y):
        encoded[i, val] = 1
    return encoded


**Write your code below this line**

--------------------------------------------

In [ ]:
def softmax(theta, X):
    """
    Function to compute associated probability for each sample and each class.

    Args:
        theta: it's the model parameter matrix. The shape is (H, K)
        X: it's the input data matrix. The shape is (N, H)

    Returns:
        softmax: it's the matrix containing probability for each sample and each class. The shape is (N, K)
    """
    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################

    return softmax


def CELoss(theta, X, y_onehot):
    """
    Function to compute softmax regression model and Cross Entropy loss.

    Args:
        theta: it's the model parameter matrix. The shape is (H, K)
        X: it's the input data matrix. The shape is (N, H)
        y_onehot: it's the label array in encoded as one hot vector. The shape is (N, K)

    Returns:
        loss: The scalar that is the mean error for each sample.
    """
    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################

    return loss


def CELoss_jacobian(theta, X, y_onehot):
    """
    Function to compute gradient of the cross entropy loss with respect the parameters.

    Args:
        theta: it's the model parameter matrix. The shape is (H, K)
        X: it's the input data matrix. The shape is (N, H)
        y_onehot: it's the label array in encoded as one hot vector. The shape is (N, K)

    Returns:
        jacobian: A matrix with the partial derivatives of the loss. The shape is (H, K)
    """
    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################

    return jacobian


def gradient_descent(theta, X, y_onehot, alpha=0.01, iterations=100):
    """
    Function to compute gradient of the cross entropy loss with respect the parameters.

    Args:
        theta: it's the model parameter matrix. The shape is (H, K)
        X: it's the input data matrix. The shape is (N, H)
        y_onehot: it's the label array in encoded as one hot vector. The shape is (N, K)
        alpha: it's the learning rate, so it determines the speed of each step of the GD algorithm
        iterations: it's the total number of step the algorithm performs

    Returns:
        theta: it's the updated matrix of the parameters after all the iterations of the optimization algorithm. The shape is (H, K)
        loss_history: it's an array with the computed loss after each iteration
    """
    # We initialize an empty array to be filled with loss value after each iteration
    loss_history = np.zeros(iterations)

    # With a for loop we compute the steps of GD algo
    for it in range(iterations):
        #####################################################
        ##                 YOUR CODE HERE                  ##
        #####################################################
        pass

    return theta, loss_history


**Do not write below this line just run it**

--------------------------------------------

*Execution can take around 10 minutes*

In [ ]:
# Evaluate the accuracy of the predictions
def compute_accuracy(predictions, true_labels):
    correct_predictions = np.sum(predictions == true_labels)
    total_predictions = len(true_labels)
    return correct_predictions / total_predictions


# Prediction function
def predict(theta, X):
    probabilities = softmax(theta, X)
    return np.argmax(probabilities, axis=1)


# Preprocess the data
X_train = [img.reshape(-1).numpy() for img, _ in train_data]
X_train = np.array(X_train)
y_train = [label for _, label in train_data]

X_test = [img.reshape(-1).numpy() for img, _ in test_data]
X_test = np.array(X_test)
y_test = [label for _, label in test_data]


# Add bias term to X
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

y_train_onehot = one_hot_encode(y_train)
y_test_onehot = one_hot_encode(y_test)


# Initialize theta
H, K = X_train.shape[1], 10  # number of features and number of classes
theta = np.random.randn(H, K) * 0.001


# Apply gradient descent to optimize theta
alpha = 0.01
iterations = 500
theta_optimized, loss_history = gradient_descent(
    theta, X_train, y_train_onehot, alpha, iterations
)


# Make predictions on the training and test data
train_predictions = predict(theta_optimized, X_train)
test_predictions = predict(theta_optimized, X_test)

train_accuracy = compute_accuracy(train_predictions, y_train)
test_accuracy = compute_accuracy(test_predictions, y_test)

# print(f"Training accuracy: {train_accuracy * 100:.2f}%")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

### **Question 3.3: Histogram Features Extraction *(2/10 Points)***

In this exercise, the goal is to implement a simple image classification procedure using the CIFAR-10 dataset. Instead of using the raw pixel values or advanced deep learning techniques, you will be extracting color histogram features from the images for classification purposes.

- Extract Color Histogram Features.
Implement the extract_histogram_features function. This function should process the dataset and convert each image into a set of histograms – one for each color channel (Red, Green, Blue).
Each histogram will serve as a feature for the classifier.

- Train a Logistic Regression Classifier.
Use the scikit-learn library to initialize and train a logistic regression classifier with the extracted histogram features.

- Evaluate the Classifier.
Use the trained classifier to predict the classes of both the training and test datasets.
Calculate and print the accuracy of the classifier for both datasets to gauge its performance.

Fill in the sections of the provided code marked with "YOUR CODE HERE". Ensure that the final pipeline is complete and functional.

In [ ]:
# Calculate pixel histograms
def extract_histogram_features(data, bins=30):
    """
    This function takes in a dataset and a number of "bins"
    and returns pixel histograms and the corresponding labels.

    Hint: Use the 'data' variable to access the images and their labels.

    Args:
        data: the input data
        bins: the number of bins

    Returns:
        histograms: the pixel histograms
        labels: the corresponding labels
    """
    #########################################
    #           INSERT YOUR CODE HERE       #
    #########################################
    return histograms, labels


**Do not write below this line just run it**

--------------------------------------------

In [ ]:
max_iter = 50

In [ ]:
train_features_hist, train_labels_hist = extract_histogram_features(train_data)
test_features_hist, test_labels_hist = extract_histogram_features(test_data)

# Train the classification model
clf = LogisticRegression(max_iter=max_iter)
clf.fit(train_features_hist, train_labels_hist)

# Performance evaluation
train_predictions_hist = clf.predict(train_features_hist)
test_predictions_hist = clf.predict(test_features_hist)

train_acc_hist = compute_accuracy(
    train_labels_hist, train_predictions_hist
)  # compute_accuracy
test_acc_hist = compute_accuracy(test_labels_hist, test_predictions_hist)

# print(f"Training accuracy: {train_acc_hist * 100:.2f}%")
print(f"Test accuracy: {test_acc_hist * 100:.2f}%")

### 3.3.2 AlexNet features extractions

Now, let's try extracting features using AlexNet, which is pretrained on ImageNet. For your convenience, we're providing the extracted features directly through this Google Drive link:
https://drive.google.com/drive/folders/12QoP_8V1hSL2bRZXvZ2sbHm8MUOCbbaP?usp=sharing.

You will need to retrain the logistic regression on these new features. After that, there will be a section on questions regarding the differing performances.


**Do not write below this line just run it**

--------------------------------------------

In [ ]:
# Load features and labels from disk
features_alex_net = "AlexNet"
train_features_alexnet = np.load(os.path.join(features_alex_net, "train_features.npy"))
train_labels_alexnet = np.load(os.path.join(features_alex_net, "train_labels.npy"))
test_features_alexnet = np.load(os.path.join(features_alex_net, "test_features.npy"))
test_labels_alexnet = np.load(os.path.join(features_alex_net, "test_labels.npy"))

# Train logistic regression
clf = LogisticRegression(max_iter=max_iter)
clf.fit(train_features_alexnet, train_labels_alexnet)

# Performance evaluation
train_predictions_alexnet = clf.predict(train_features_alexnet)
train_acc_alexnet = compute_accuracy(train_labels_alexnet, train_predictions_alexnet)

test_predictions_alexnet = clf.predict(test_features_alexnet)
test_acc_alexnet = compute_accuracy(test_labels_alexnet, test_predictions_alexnet)

# print(f"Training accuracy: {train_acc_alexnet * 100:.2f}%")
print(f"Test accuracy: {test_acc_alexnet * 100:.2f}%")

### 3.3.3 Performance Evaluation

Now, we aim to compare the performances obtained.
Please note that this section is purely theoretical, you should execute the code provided below and then answer the questions that will be presented at the end of this section.

**Do not write below this line just run it**

--------------------------------------------

In [ ]:
# Compute diffusion matrix for Histogram Features example
conf_matrix_hist = confusion_matrix(test_labels_hist, test_predictions_hist)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_hist, annot=True, fmt="g", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Histogram features based Performance")
plt.show()

# Compute diffusion matrix for AlexNet Features example
conf_matrix_alexnet = confusion_matrix(test_labels_alexnet, test_predictions_alexnet)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_alexnet, annot=True, fmt="g", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Alexnet features based Performance")
plt.show()


**Answer the following questions**

--------------------------------------------

1. Provide a comment on the confusion matrix and try to give an explanation for why this happens?
2. Why is the accuracy higher when using raw pixels as input compared to when we use features extracted from the histogram?


**Write your answer below this line**

--------------------------------------------


1.
2.

--------------------------------------------

## **Question 4 (BONUS): Trasfer Learning on CIFAR-10 _(5 points)_**

**Transfer Learning (TL)** is a machine learning technique that consists in reusing a pre-trained model and its weights to perform a task similar to the one the model has been trained on. Pre-trained models are usually trained using large amounts of data and are really useful if you have just a small dataset to learn from.

In this bonus question you are going to leverage a pre-trained DNN, [**AlexNet**](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf), trained on the IMAGENET dataset, in order to train two classification models on a different, smaller, dataset (CIFAR-10 in this case).

The dataset your are going to use is made of 50000 feature embeddings with 1024 dimensions obtained by encoding the images in CIFAR-10 with the pre-trained AlexNet. This means that your classifier is going to leverage the representation power of a relatively large DNN without the actual need to train in from scratch, which would require high GPU power and many hours/days of training.

For the most curious among you: at the end of the exercise you will find the actual code that we used to encode the CIFAR-10 dataset with AlexNet using the [**PyTorch**](https://pytorch.org/) framework. In case you wanted to run it, we suggest to either use Google Colab with the runtime set to GPU (Runtime -> Change runtime type -> GPU) or a PC/laptop with a relative capable GPU (minimum 4GB of VRAM).

**Do not write below this line, just run it**

--------------------------------------------

In [ ]:
# import libraries and set seed
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

np.random.seed(123)

In [ ]:
# load the train and test datasets
X_train = np.load("./data/processed_data/X_train.npy")
y_train = np.load("./data/processed_data/y_train.npy")
y_onehot_train = np.load("./data/processed_data/y_onehot_train.npy")

X_test = np.load("./data/processed_data/X_test.npy")
y_test = np.load("./data/processed_data/y_test.npy")

In [ ]:
# Shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("y_onehot_train shape:", y_onehot_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

### **Question 4.1: Train a KNN Classifier on CIFAR-10 _(2/5 points)_**

Your goal is to train a KNN Classification model using the pre-processed and encoded CIFAR-10 dataset (given by us).

The encoded dataset is made of 50000 feature embeddings with 1024 dimensions. The labels are encoded as integers in the range [0,9], they will serve as the target of your classifier.

KNN is a simple and intuitive classification algorithm that works by assigning a label to a new sample based on the labels of the K nearest samples in the training set. The label is the one that appears the most among the K nearest samples.


**HINTS:**
* You will have to train the KNN classifier on the training set by trying different values of K. At the end you will have to report the accuracy of your model on the test set for the best value of K.
* You **must** use sklearn.neighbors.KNeighborsClassifier to train your model. You can find the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
# subsampling data
num_training = 10000
mask = list(range(num_training))
X_train_sub = X_train[mask]
y_train_sub = y_train[mask]

In [ ]:
k_range = range(5, 11)
scores = {}
scores_list = []

**Write your code below this line**

--------------------------------------------

In [ ]:
##############################################
#           INSERT YOUR CODE HERE             #
##############################################

**Do not write below this line, just run it**

--------------------------------------------

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(k_range, scores_list)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

**Do not write below this line, just run it**

--------------------------------------------

In [ ]:
# now pick the best k and train on the whole training set
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
# print the accuracy
print(metrics.accuracy_score(y_test, y_pred))

### **Question 4.2: Train a Softmax Classifier on CIFAR-10 _(2/5 points)_**

Your goal is to train a Softmax Regression Model using the pre-processed and encoded CIFAR-10 dataset (given by us) and the code written by you in Question 3.2.

Here you have to write the code to train your Softmax Regression Model.  
Your function has to return the inital theta, the learned theta, and the loss history.  
You can evaluate its performance with the code below.

**HINT:** Experiment with different alpha's with a small number of iterations to keep the training time low. Once you find the best alpha you can train your model for as long as needed.

**Write your code below this line**

--------------------------------------------

In [ ]:
def fit_softmax_classifier(X_train, y_onehot_train, alpha, n_iter):
    #####################################################
    ##                 YOUR CODE HERE                  ##
    #####################################################

    return theta0, theta_final, log_l_history

**Do not write below this line, just run it**

--------------------------------------------

In [ ]:
theta0, theta_final, log_l_history = fit_softmax_classifier(
    X_train, y_onehot_train, alpha=1e4, n_iter=1000
)

In [ ]:
fig, ax = plt.subplots(num=2)

ax.set_ylabel("loss")
ax.set_xlabel("Iterations")
_ = ax.plot(range(len(log_l_history)), log_l_history, "b.")

In [ ]:
def compute_accuracy(theta, X, y):
    """
    Function to compute accuracy metrics of the softmax regression model.

    Input:
    theta: it's the final parameter matrix. The one we learned after all the iterations of the GD algorithm. The shape is (H, K)
    X: it's the input data matrix. The shape is (N, H)
    y: it's the label array. The shape is (N, 1)

    Output:
    accuracy: Score of the accuracy.
    """

    return accuracy

**Do not write below this line, just run it**

--------------------------------------------

In [ ]:
theta0 = np.random.rand(X_train.shape[1], 10)

acc_rand_train = compute_accuracy(theta0, X_train, y_train)
acc_train = compute_accuracy(theta_final, X_train, y_train)

acc_rand_test = compute_accuracy(theta0, X_test, y_test)
acc_test = compute_accuracy(theta_final, X_test, y_test)

print(f"Accuracy with random parameters on train set: {acc_rand_train}")
print(f"Accuracy with learned parameters on train set: {acc_train}")
print(f"Accuracy with random parameters on test set: {acc_rand_test}")
print(f"Accuracy with learned parameters on test set: {acc_test}")

### **Question 4.3: Report _(1/5 points)_**

Write a report comparing the results obtained with the two different classifiers.

Try to answer the following questions:
- Which classifier performs better? Why? Is there a trade-off between the two in terms of perfomance/computation time?
- Do the classifiers obtain the same accuracy on train and test sets? If not, try to give an explanation.
- What values of alpha are best to train the two classifiers and what is the optimal number of iterations?

--------------------------------------------
**Write your report here**


--------------------------------------------

### **Code used to create the dataset _(no points)_**

This part of the homework will not give you any points and it is not needed to complete the assignment, so please don't ask for help before the deadline if you are not able to run it.

Run only if you have the following minimum requirements:
- Google Colab with GPU runtime active (Runtime -> Change runtime type -> GPU) or a PC/laptop with a relatively capable GPU (minimum 4GB of VRAM)
- High speed connection and minimum 4GB of free storage
    - the download of the dataset and the weights of the pre-trained AlexNet have large sizes!

In [ ]:
# import useful libraries and functions
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

from sklearn.preprocessing import normalize
from tqdm import tqdm

torch.manual_seed(123)
np.random.seed(123)

In [ ]:
# image transform for CIFAR-10 dataset
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# download CIFAR-10 dataset for train and test
train_data = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
test_data = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

# download pre-trained AlexNet
pretrained_net = torch.hub.load(
    "pytorch/vision:v0.10.0", "alexnet", weights="AlexNet_Weights.DEFAULT"
)
classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

# define the device you will use for training (cpu or cuda)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Let's take a look at it!

The network is composed by a backbone with multiple Conv2d, ReLU and MaxPool2d layers, by an average pooling layer, and by a classifier.

We are going to use just the backbone of the pre-trained AlexNet (the layers included into *(features)*) and we will define a new classifier. We will train everything together on the CIFAR-10 dataset and we will leverage the pre-trained weights of the backbone to obtain high classification accuracy with just a few epochs of training. This is the power of Transfer Learning!

In [ ]:
pretrained_net

In [ ]:
# create train and test dataloaders
trainloader = torch.utils.data.DataLoader(
    train_data, batch_size=256, shuffle=True, num_workers=2
)
testloader = torch.utils.data.DataLoader(
    test_data, batch_size=256, shuffle=True, num_workers=2
)

In [ ]:
# define the encoder network to obtain image embeddings
encoder = pretrained_net.features.to(device).eval()
encoder[12] = nn.MaxPool2d(kernel_size=9, stride=4)
encoder

In [ ]:
def create_dataset(loader, feat_size, num_classes):
    X = np.empty((0, feat_size))
    y_onehot = np.empty((0, num_classes))

    for i, data in tqdm(enumerate(loader), total=len(loader)):
        images, labels = data[0].to(device), data[1].to(device)
        emb = encoder(images)
        emb = torch.flatten(emb, 1).detach().cpu().numpy()
        X = np.concatenate((X, emb))
        y_temp = nn.functional.one_hot(labels, 10).detach().cpu().numpy()
        y_onehot = np.concatenate((y_onehot, y_temp))

    X = normalize(X, axis=1, norm="l1")
    y = np.argmax(y_onehot, axis=1)
    return X, y, y_onehot

In [ ]:
print("Preparing train data...")
X_train, y_train, y_onehot_train = create_dataset(trainloader, 1024, 10)
print("Preparing test data...")
X_test, y_test, _ = create_dataset(testloader, 1024, 10)

In [ ]:
if not os.path.exists("./data/processed_data"):
    os.makedirs("./data/processed_data")

np.save("./data/processed_data/X_train.npy", X_train)
np.save("./data/processed_data/y_train.npy", y_train)
np.save("./data/processed_data/y_onehot_train.npy", y_onehot_train)

np.save("./data/processed_data/X_test.npy", X_test)
np.save("./data/processed_data/y_test.npy", y_test)